In [39]:
import gzip
import numpy as np
import random
from collections import defaultdict

In [12]:
def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [42]:
# Visit prediction
# 1

In [150]:
# 100,000 sets for training and another 100,000 for validation
businessCount = defaultdict(int)
totalPurchases = 0
visitVal = []

for l in readGz("train.json.gz"):
    if (totalPurchases < 100000):
        user,business = l['userID'],l['businessID']
        businessCount[business] += 1
        totalPurchases += 1
    else:
        user,business = l['userID'],l['businessID']
        visitVal.append((user, business, 1))
    

In [152]:
# create 100,000 more non-visited data points


visited = set()
uniqueUser = set()
uniqueBuss = set()

for u,b,c in visitVal:
    uniqueUser.add(u)
    uniqueBuss.add(b)
    visited.add((u,b))

for l in range(0,100000):
    user = random.sample(uniqueUser, 1)[0]
    business = random.sample(uniqueBuss, 1)[0]
    while ((user,business) in visited):
        user = random.sample(uniqueUser, 1)[0]
        business = random.sample(uniqueBuss, 1)[0]
    visitVal.append((user,business, 0))

In [154]:
len(visitVal)

200000

In [155]:
mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

In [157]:
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/2: break

correct = 0
index = 0

for u,b,c in visitVal:
    if (((b in return1) and (c==1)) or ((b not in return1) and (c==0))) :
        correct += 1
    index += 1
    
accuracy = float(correct)/len(visitVal)
print accuracy

0.621255


In [119]:
#2

In [158]:
# Try 75th percentile
return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPurchases/4: break

correct = 0
index = 0

for u,b,c in visitVal:
    if (((b in return1) and (c==1)) or ((b not in return1) and (c==0))) :
        correct += 1
    index += 1
    
accuracy = float(correct)/len(visitVal)
print accuracy

0.58301


In [162]:
# Try 25th percentile

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > 3*totalPurchases/4: break

correct = 0
index = 0

for u,b,c in visitVal:
    if (((b in return1) and (c==1)) or ((b not in return1) and (c==0))) :
        correct += 1
    index += 1
    
accuracy = float(correct)/len(visitVal)
print accuracy

0.60618


In [171]:
# Try 40th percentile

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > 6*totalPurchases/10: break

correct = 0
index = 0

for u,b,c in visitVal:
    if (((b in return1) and (c==1)) or ((b not in return1) and (c==0))) :
        correct += 1
    index += 1
    
accuracy = float(correct)/len(visitVal)
print accuracy

0.622945


In [172]:
# Try 60th percentile

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > 4*totalPurchases/10: break

correct = 0
index = 0

for u,b,c in visitVal:
    if (((b in return1) and (c==1)) or ((b not in return1) and (c==0))) :
        correct += 1
    index += 1
    
accuracy = float(correct)/len(visitVal)
print accuracy

0.61187


In [173]:
# It seems that 50th percentile gives the highest acuracy and although the 40th percentile test give slightly better, 
# I believe this is due to random chance. I think it is the best because  

In [174]:
# 3 
# Users may tend to repeatedly visit business of the same type. Build a baseline that returns ‘True’ if
# a user has visited a business of the same category before (at least one category in common), or zero
# otherwise

In [328]:
# 100,000 sets for training and another 100,000 for validation
businessCount = defaultdict(int)
totalPurchases = 0
visitVal = []
userCats = {}

for l in readGz("train.json.gz"):
    if (totalPurchases < 100000):
        user,business,categories = l['userID'],l['businessID'],l['categories']
        userCats[user] = list(set(userCats.get(user,[])).union(categories))
        
    else:
        user,business,categories = l['userID'],l['businessID'],l['categories']
        visitVal.append((user, business, categories, 1))
            
    totalPurchases += 1

In [329]:
userCats['U600574911']

[u'Donut Shop',
 u'European Restaurant',
 u'Chinese Restaurant',
 u'Middle Eastern Restaurant',
 u'Dessert Shop',
 u'Asian Restaurant',
 u'Ice Cream Shop']

In [330]:
# create 100,000 more non-visited data points

visited = set()
uniqueUser = set()
uniqueBuss = set()
bussCat = {}

for u,b,c,d in visitVal:
    uniqueUser.add(u)
    uniqueBuss.add(b)
    visited.add((u,b))
    bussCat[b] = c
    

In [331]:
for l in range(0,100000):
    user = random.sample(uniqueUser, 1)[0]
    bussiness = random.sample(uniqueBuss, 1)[0]
  
    while ((user,business) in visited):
        user = random.sample(uniqueUser, 1)[0]
        business = random.sample(uniqueBuss, 1)[0]
        
    visitVal.append((user,business, bussCat[business], 0))
    

In [332]:
visitVal[100]

('U237829706', 'B556394301', [u'Restaurant', u'Soul Food Restaurant'], 1)

In [333]:
visitVal[100001]

('U932542850', 'B589288198', [u'American Restaurant'], 0)

In [334]:
# Test the accuracy

predicts = []
for user,business,cat,d in visitVal:
    cat1 = userCats.get(user,[])

    intersection = list(set(cat1) & set(cat))
    
    if intersection:
        predicts.append(1)
    else:
        predicts.append(0)

accuracy = [(a[3]==b) for (a,b) in zip(visitVal,predicts)]
print 1.0*sum(accuracy)/len(accuracy)

0.66293


In [339]:
predictions = open("predictions_Visit.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue

    u,i = l.strip().split('-')
    cat1 = userCats.get(u,[])
    cat2 = bussCat.get(i,[])
    intersection = list(set(cat1) & set(cat2))
    
    if intersection:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

In [340]:
# 4
# I uploaded my solution to Kaggle and got a score of 0.64930
# My user name is David Dinata

In [341]:
# Category prediction
# 5